# INDEX
* [Imports and functions](#Imports-and-functions)
* [Configuration](#Configuration)
* [Prepare dataset](#Prepare-dataset)
* [Build model](#Build-model)
    * [Model inputs](#Model-inputs)
    * [Model output](#Model-output)
    * [Model](#Model)
* [Train model](#Train-model)
    * [Training stats](#Training-stats)
* [Save model and resources](#Save-model-and-resources)
* [Test model](#Test-model)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi

Thu Jan  5 12:01:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    39W / 300W |      0MiB / 16384MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
# eventual initialization for colab notebooks
if IN_COLAB:
  # we try hard to be re-entrant,
  # that is to be able to rerun this without cloning repository more than once
  COLAB_BRANCH = "master"
  !curl https://raw.githubusercontent.com/openfoodfacts/off-category-classification/$COLAB_BRANCH/lib/colab.py --output /content/colab.py
  !cd /content && python /content/colab.py $COLAB_BRANCH
  %cd /content/off-category-classification/experiments

In [4]:
# codecarbon - start tracking
from codecarbon import EmissionsTracker

tracker = EmissionsTracker(log_level="WARNING", save_to_api=True, experiment_id="6d2c8401-afba-42de-9600-6e95bea5fd80")
tracker.start()

[codecarbon WARNING @ 12:01:25] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 12:01:26] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.


# Imports

In [5]:
import sys
from pathlib import Path

PROJECT_DIR = Path("..").absolute().resolve()
sys.path.append(str(PROJECT_DIR)) # append a relative path to the top package to the search path

In [6]:
import dataclasses
import random

import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from tensorflow.keras import callbacks, layers
from tensorflow.keras.utils import plot_model

from lib.dataset import filter_empty_labels, flat_batch, get_vocabulary, load_dataset, select_feature
from lib.directories import init_cache_dir, init_model_dir
from lib.io import load_model, save_model
from lib.model import top_labeled_predictions, top_predictions_table
from lib.plot import plot_training_stats
from lib.taxonomy import get_taxonomy

/home/raphael/off-category-classification/.venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Configuration

In [7]:
MODEL_BASE_DIR = PROJECT_DIR / "model"
CACHE_DIR = PROJECT_DIR / "tensorflow_cache"

PREPROC_BATCH_SIZE = 10_000  # some large value, only affects execution time

# splits are handled by `tfds.load`, see doc for more elaborate ways to sample
TRAIN_SPLIT = 'train[:80%]'
VAL_SPLIT = 'train[80%:90%]'
TEST_SPLIT = 'train[90%:]'

@dataclasses.dataclass
class Config:
    mixed_precision: bool = False
    random_seed: int = 42


CONFIG = Config(mixed_precision=False)

# Prepare dataset

Run this once to fetch, build and cache the dataset.
Further runs will be no-ops, unless you force operations (see TFDS doc).

Once this is done, `load_dataset('off_categories', ...)` to access the dataset.

In [8]:
category_taxonomy = get_taxonomy("category", offline=True)
ingredient_taxonomy = get_taxonomy("ingredient", offline=True)

In [ ]:
import datasets.off_categories

builder = tfds.builder('off_categories')
builder.download_and_prepare()

# Or run via command line (if `tfds` is in the path):
# !cd ../datasets && tfds build off_categories

Dl Completed...: 0 url [00:00, ? url/s]
Generating splits...:   0%|                                                                                                                  | 0/1 [00:00<?, ? splits/s]
Generating train examples...: 0 examples [00:00, ? examples/s]
Generating train examples...: 276 examples [00:00, 2750.55 examples/s]
Generating train examples...: 552 examples [00:00, 2536.95 examples/s]
Generating train examples...: 807 examples [00:00, 2435.18 examples/s]
Generating train examples...: 1052 examples [00:00, 2280.30 examples/s]
Generating train examples...: 1317 examples [00:00, 2401.64 examples/s]
Generating train examples...: 1559 examples [00:00, 2337.12 examples/s]
Generating train examples...: 1796 examples [00:00, 2343.18 examples/s]
Generating train examples...: 2032 examples [00:00, 2324.15 examples/s]
Generating train examples...: 2265 examples [00:00, 2300.31 examples/s]
Generating train examples...: 2506 examples [00:01, 2332.81 examples/s]
Generating 

Generating train examples...: 22692 examples [00:09, 2311.99 examples/s]
Generating train examples...: 22925 examples [00:09, 2297.20 examples/s]
Generating train examples...: 23167 examples [00:09, 2331.18 examples/s]
Generating train examples...: 23403 examples [00:09, 2338.62 examples/s]
Generating train examples...: 23638 examples [00:10, 2339.46 examples/s]
Generating train examples...: 23873 examples [00:10, 2257.65 examples/s]
Generating train examples...: 24120 examples [00:10, 2315.61 examples/s]
Generating train examples...: 24355 examples [00:10, 2323.22 examples/s]
Generating train examples...: 24588 examples [00:10, 2273.09 examples/s]
Generating train examples...: 24849 examples [00:10, 2370.11 examples/s]
Generating train examples...: 25096 examples [00:10, 2398.74 examples/s]
Generating train examples...: 25337 examples [00:10, 2279.46 examples/s]
Generating train examples...: 25567 examples [00:10, 2260.07 examples/s]
Generating train examples...: 25795 examples [00:10

Generating train examples...: 49309 examples [00:20, 2277.40 examples/s]
Generating train examples...: 49545 examples [00:20, 2284.09 examples/s]
Generating train examples...: 49774 examples [00:21, 2219.78 examples/s]
Generating train examples...: 49997 examples [00:21, 2144.79 examples/s]
Generating train examples...: 50216 examples [00:21, 2156.03 examples/s]
Generating train examples...: 50447 examples [00:21, 2200.59 examples/s]
Generating train examples...: 50668 examples [00:21, 2002.77 examples/s]
Generating train examples...: 50901 examples [00:21, 2088.69 examples/s]
Generating train examples...: 51114 examples [00:21, 2078.63 examples/s]
Generating train examples...: 51341 examples [00:21, 2132.81 examples/s]
Generating train examples...: 51558 examples [00:21, 2143.31 examples/s]
Generating train examples...: 51800 examples [00:21, 2223.92 examples/s]
Generating train examples...: 52064 examples [00:22, 2345.44 examples/s]
Generating train examples...: 52325 examples [00:22

Generating train examples...: 76052 examples [00:32, 2732.85 examples/s]
Generating train examples...: 76329 examples [00:32, 2690.71 examples/s]
Generating train examples...: 76601 examples [00:32, 2674.85 examples/s]
Generating train examples...: 76871 examples [00:32, 2627.68 examples/s]
Generating train examples...: 77135 examples [00:32, 2504.30 examples/s]
Generating train examples...: 77388 examples [00:32, 2365.75 examples/s]
Generating train examples...: 77638 examples [00:32, 2401.73 examples/s]
Generating train examples...: 77880 examples [00:32, 2276.12 examples/s]
Generating train examples...: 78119 examples [00:32, 2307.29 examples/s]
Generating train examples...: 78352 examples [00:33, 2242.61 examples/s]
Generating train examples...: 78594 examples [00:33, 2290.95 examples/s]
Generating train examples...: 78825 examples [00:33, 2259.74 examples/s]
Generating train examples...: 79052 examples [00:33, 2208.03 examples/s]
Generating train examples...: 79298 examples [00:33

Generating train examples...: 102181 examples [00:43, 2425.26 examples/s]
Generating train examples...: 102424 examples [00:43, 2380.08 examples/s]
Generating train examples...: 102663 examples [00:43, 2366.69 examples/s]
Generating train examples...: 102906 examples [00:43, 2383.38 examples/s]
Generating train examples...: 103145 examples [00:43, 2363.30 examples/s]
Generating train examples...: 103382 examples [00:43, 2308.32 examples/s]
Generating train examples...: 103614 examples [00:43, 2297.54 examples/s]
Generating train examples...: 103847 examples [00:43, 2306.23 examples/s]
Generating train examples...: 104078 examples [00:44, 2254.56 examples/s]
Generating train examples...: 104304 examples [00:44, 2254.91 examples/s]
Generating train examples...: 104530 examples [00:44, 2231.94 examples/s]
Generating train examples...: 104754 examples [00:44, 2204.38 examples/s]
Generating train examples...: 104975 examples [00:44, 2132.09 examples/s]
Generating train examples...: 105201 e

...

Generating train examples...: 521884 examples [03:27, 2080.90 examples/s]
Generating train examples...: 522093 examples [03:27, 2040.57 examples/s]
Generating train examples...: 522298 examples [03:28, 1999.33 examples/s]
Generating train examples...: 522502 examples [03:28, 2008.33 examples/s]
Generating train examples...: 522744 examples [03:28, 2127.39 examples/s]
Generating train examples...: 522992 examples [03:28, 2230.40 examples/s]
Generating train examples...: 523217 examples [03:28, 2234.69 examples/s]
Generating train examples...: 523441 examples [03:28, 2213.89 examples/s]
Generating train examples...: 523663 examples [03:28, 2175.33 examples/s]
Generating train examples...: 523881 examples [03:28, 2136.01 examples/s]
Generating train examples...: 524095 examples [03:28, 2095.28 examples/s]
Generating train examples...: 524305 examples [03:28, 2083.78 examples/s]
Generating train examples...: 524534 examples [03:29, 2143.46 examples/s]
Generating train examples...: 524758 e

Generating train examples...: 552348 examples [03:38, 2929.81 examples/s]
Generating train examples...: 552642 examples [03:38, 2897.52 examples/s]
Generating train examples...: 552933 examples [03:38, 2894.15 examples/s]
Generating train examples...: 553223 examples [03:38, 2895.72 examples/s]
Generating train examples...: 553531 examples [03:39, 2948.46 examples/s]
Generating train examples...: 553827 examples [03:39, 2902.36 examples/s]
Generating train examples...: 554121 examples [03:39, 2913.14 examples/s]
Generating train examples...: 554414 examples [03:39, 2915.81 examples/s]
Generating train examples...: 554706 examples [03:39, 2911.39 examples/s]
Generating train examples...: 555019 examples [03:39, 2974.97 examples/s]
Generating train examples...: 555320 examples [03:39, 2978.60 examples/s]
Generating train examples...: 555629 examples [03:39, 3010.75 examples/s]
Generating train examples...: 555935 examples [03:39, 3024.48 examples/s]
Generating train examples...: 556238 e

Generating train examples...: 583808 examples [03:49, 2964.22 examples/s]
Generating train examples...: 584105 examples [03:49, 2942.53 examples/s]
Generating train examples...: 584400 examples [03:49, 2913.21 examples/s]
Generating train examples...: 584692 examples [03:49, 2889.54 examples/s]
Generating train examples...: 584989 examples [03:49, 2911.75 examples/s]
Generating train examples...: 585294 examples [03:50, 2943.71 examples/s]
Generating train examples...: 585589 examples [03:50, 2923.12 examples/s]
Generating train examples...: 585896 examples [03:50, 2965.90 examples/s]
Generating train examples...: 586193 examples [03:50, 2965.13 examples/s]
Generating train examples...: 586501 examples [03:50, 2995.88 examples/s]
Generating train examples...: 586801 examples [03:50, 2943.29 examples/s]
Generating train examples...: 587099 examples [03:50, 2951.90 examples/s]
Generating train examples...: 587395 examples [03:50, 2850.45 examples/s]
Generating train examples...: 587681 e

Generating train examples...: 617161 examples [04:00, 3026.16 examples/s]
Generating train examples...: 617488 examples [04:00, 3096.94 examples/s]
Generating train examples...: 617802 examples [04:00, 3103.14 examples/s]
Generating train examples...: 618113 examples [04:00, 3019.34 examples/s]
Generating train examples...: 618416 examples [04:00, 3002.92 examples/s]
Generating train examples...: 618718 examples [04:00, 3004.95 examples/s]
Generating train examples...: 619020 examples [04:00, 3008.85 examples/s]
Generating train examples...: 619335 examples [04:01, 3049.27 examples/s]
Generating train examples...: 619642 examples [04:01, 3048.41 examples/s]
Generating train examples...: 619950 examples [04:01, 3057.64 examples/s]
Generating train examples...: 620264 examples [04:01, 3081.76 examples/s]
Generating train examples...: 620573 examples [04:01, 3069.79 examples/s]
Generating train examples...: 620881 examples [04:01, 2972.08 examples/s]
Generating train examples...: 621179 e

Generating train examples...: 648788 examples [04:11, 2641.07 examples/s]
Generating train examples...: 649053 examples [04:11, 2629.51 examples/s]
Generating train examples...: 649416 examples [04:11, 2919.92 examples/s]
Generating train examples...: 649863 examples [04:11, 3373.62 examples/s]
Generating train examples...: 650203 examples [04:11, 3217.47 examples/s]
Generating train examples...: 650528 examples [04:11, 3055.84 examples/s]
Generating train examples...: 650837 examples [04:12, 2983.10 examples/s]
Generating train examples...: 651138 examples [04:12, 2950.73 examples/s]
Generating train examples...: 651435 examples [04:12, 2808.98 examples/s]
Generating train examples...: 651718 examples [04:12, 2670.67 examples/s]
Generating train examples...: 652001 examples [04:12, 2713.28 examples/s]
Generating train examples...: 652275 examples [04:12, 2564.19 examples/s]
Generating train examples...: 652564 examples [04:12, 2648.90 examples/s]
Generating train examples...: 652847 e

# Build model

In [ ]:
def set_random_seed(seed):
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_random_seed(CONFIG.random_seed)

if CONFIG.mixed_precision:
    print("Using mixed precision")
    tf.keras.mixed_precision.set_global_policy("mixed_float16")

## Model inputs

In [ ]:
# we use dicts so rerunning individual model cells is idempotent
inputs = {}
input_graphs = {}

In [ ]:
ds = load_dataset('off_categories', split=TRAIN_SPLIT)

In [ ]:
%%time

feature_name = 'product_name'

product_name_input = tf.keras.Input(shape=(1,), dtype=tf.string, name=feature_name)

product_name_vectorizer = layers.TextVectorization(
    split = 'whitespace',
    max_tokens = 93_000,
    output_sequence_length = 30)

product_name_vectorizer.adapt(
    select_feature(ds, feature_name).batch(PREPROC_BATCH_SIZE))

x = product_name_vectorizer(product_name_input)

x = layers.Embedding(
    input_dim = product_name_vectorizer.vocabulary_size(),
    output_dim = 64,
    mask_zero = False)(x)

product_name_graph = layers.Bidirectional(layers.LSTM(
    units = 64,
    recurrent_dropout = 0.0,
    dropout = 0.2))(x)

inputs[feature_name] = product_name_input
input_graphs[feature_name] = product_name_graph

len(product_name_vectorizer.get_vocabulary())

In [ ]:
%%time

feature_name = 'ingredients_tags'

ingredients_input = tf.keras.Input(shape=(None,), dtype=tf.string, name=feature_name)

ingredients_vocab = get_vocabulary(
    flat_batch(select_feature(ds, feature_name), batch_size=PREPROC_BATCH_SIZE),
    min_freq = 3,
    max_tokens = 5_000)

ingredients_graph = layers.StringLookup(
    vocabulary = ingredients_vocab,
    output_mode = 'multi_hot')(ingredients_input)

inputs[feature_name] = ingredients_input
input_graphs[feature_name] = ingredients_graph

len(ingredients_vocab)

## Model output

In [ ]:
%%time

labels = 'categories_tags'

categories_vocab = get_vocabulary(
    flat_batch(select_feature(ds, labels), batch_size=PREPROC_BATCH_SIZE),
    min_freq = 10)

# StringLookup(output_mode='multi_hot') mode requires num_oov_indices >= 1.
# We don't want OOVs in the categories_tags output layer, since it wouldn't make
# sense to predict OOV. So we'll drop the OOV in _transform below.
# Be careful when using StringLookup methods, some of them will return values
# based on a vocabulary with OOV (e.g. vocabulary_size()). Keep this in mind when
# mapping predictions back to the original vocabulary.
categories_multihot = layers.StringLookup(
    vocabulary = categories_vocab,
    output_mode = 'multi_hot',
    num_oov_indices = 1)

def categories_encode(ds: tf.data.Dataset):
    @tf.function
    @tf.autograph.experimental.do_not_convert
    def _transform(x, y):
        y = categories_multihot(y)
        y = y[1:]  # drop OOV
        return (x, y)

    # applies to non-batched dataset
    return (
        ds
        .map(_transform, num_parallel_calls=tf.data.AUTOTUNE, deterministic=True)
        .apply(filter_empty_labels)
    )

len(categories_vocab)

## Model

In [ ]:
# ensure final order is independent of cell execution/insertion order
features = sorted(inputs.keys())

x = layers.Concatenate()([input_graphs[k] for k in features])
x = layers.Dropout(0.2)(x)
x = layers.Dense(64)(x)
x = layers.Dropout(0.2)(x)
x = layers.Activation('relu')(x)
output = layers.Dense(len(categories_vocab), activation='sigmoid')(x)

model = tf.keras.Model(inputs=[inputs[k] for k in features], outputs=[output])

threshold = 0.5
num_labels = len(categories_vocab)

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.0),
    metrics = [
        tf.metrics.Precision(thresholds=threshold, name='precision'),
        tf.metrics.Recall(thresholds=threshold, name='recall'),
        tfa.metrics.F1Score(average='micro', threshold=threshold, num_classes=num_labels, name='f1_score_micro'),
        tfa.metrics.F1Score(average='macro', threshold=threshold, num_classes=num_labels, name='f1_score_macro')
    ]
)

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

# Train model

In [ ]:
%%time

# Remember to clean obsolete dirs once in a while
MODEL_DIR = init_model_dir(MODEL_BASE_DIR)
CACHE_DIR = init_cache_dir(CACHE_DIR)

batch_size = 128

ds_train = (
    load_dataset('off_categories', split=TRAIN_SPLIT, features=features, as_supervised=True)
    .apply(categories_encode)
    .padded_batch(batch_size)
    .cache(str(CACHE_DIR / 'train'))
)

ds_val = (
    load_dataset('off_categories', split=VAL_SPLIT, features=features, as_supervised=True)
    .apply(categories_encode)
    .padded_batch(batch_size)
    .cache(str(CACHE_DIR / 'val'))
)

history = model.fit(
    ds_train,
    epochs = 50,
    validation_data = ds_val,
    callbacks = [
        callbacks.TerminateOnNaN(),
        callbacks.ModelCheckpoint(
            filepath = str(MODEL_DIR / "weights.{epoch:02d}-{val_loss:.4f}"),
            monitor = 'val_loss',
            save_best_only = True,
            save_format = 'tf',
        ),
        callbacks.EarlyStopping(monitor='val_loss', patience=4),
        callbacks.CSVLogger(str(MODEL_DIR / 'training.log')),
        callbacks.History(),
        callbacks.TensorBoard(log_dir='./tensorboard_logs'),
    ]
)

## Training stats

In [ ]:
stats = pd.read_csv(MODEL_DIR / 'training.log')
stats

In [ ]:
plot_training_stats(stats)

# Save model and resources

In [ ]:
SAVED_MODEL_DIR = MODEL_DIR / 'saved_model'

@tf.function
def serving_func(*args, **kwargs):
    preds = model(*args, **kwargs)
    return top_labeled_predictions(preds, categories_vocab, k=50)

save_model(SAVED_MODEL_DIR, model, categories_vocab, serving_func)

# Test model

In [ ]:
m, labels = load_model(SAVED_MODEL_DIR)

In [ ]:
ds_test = load_dataset('off_categories', split=TEST_SPLIT)

In [ ]:
%%time

preds_test = m.predict(ds_test.padded_batch(128))
preds_test

In [ ]:
# This is the function exported as the default serving function in our saved model
top_preds_test = top_labeled_predictions(preds_test, labels, k=3)
top_preds_test

In [ ]:
# Same data, but pretty
pred_table_test = top_predictions_table(top_preds_test)

# Add some interpretable features to the final table
# Table must be row-aligned with predictions above (= taken from same data sample)
extra_cols_test = as_dataframe(select_features(ds_test, ['code', 'product_name']))

pd.concat([extra_cols_test, pred_table_test], axis=1)

In [ ]:
# codecarbon - stop tracking
tracker.stop()